# SpamDetector

First we will rebuild the Algo Bag of Words in separate Steps to represent the words in the coloums with numbers. Reason: just numbers can handeld by an learning-algorithm.

In [34]:
import pandas as pd

In [35]:
df = pd.read_table('smsspamcollection/SMSSpamCollection',
                   sep='\t', 
                   header=None, 
                   names=['label', 'sms_message'])
print(df.head())

  label                                        sms_message
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [36]:
# map ham and spam to 0/1 to deal with it
df['label'] = df.label.map({'ham':0, 'spam':1})
print(df.shape)
print(df.head())

(5572, 2)
   label                                        sms_message
0      0  Go until jurong point, crazy.. Available only ...
1      0                      Ok lar... Joking wif u oni...
2      1  Free entry in 2 a wkly comp to win FA Cup fina...
3      0  U dun say so early hor... U c already then say...
4      0  Nah I don't think he goes to usf, he lives aro...


In [37]:
print("Convert to lower case")
documents = ['Hello, how are you!',
             'Win money, win from home.',
             'Call me now.',
             'Hello, Call hello you tomorrow?']
print(documents)
lower_case_documents = []
for i in documents:
    lower_case_documents.append(i.lower())
print(lower_case_documents)

Convert to lower case
['Hello, how are you!', 'Win money, win from home.', 'Call me now.', 'Hello, Call hello you tomorrow?']
['hello, how are you!', 'win money, win from home.', 'call me now.', 'hello, call hello you tomorrow?']


In [38]:
print("Remove all punctation")
import string
without_punktation = []
for i in lower_case_documents:
    without_punktation.append(i.translate(str.maketrans('', '', string.punctuation)))
print(without_punktation)

Remove all punctation
['hello how are you', 'win money win from home', 'call me now', 'hello call hello you tomorrow']


In [39]:
print("Split into single words")
words = []
for i in without_punktation:
    words.append(i.split(' '))
print(words)

gezaehlte_Liste = []
import pprint
from collections import Counter
for i in words:
    counts = Counter(i)
    gezaehlte_Liste.append(counts)
pprint.pprint(gezaehlte_Liste)

Split into single words
[['hello', 'how', 'are', 'you'], ['win', 'money', 'win', 'from', 'home'], ['call', 'me', 'now'], ['hello', 'call', 'hello', 'you', 'tomorrow']]
[Counter({'hello': 1, 'are': 1, 'how': 1, 'you': 1}),
 Counter({'win': 2, 'money': 1, 'from': 1, 'home': 1}),
 Counter({'call': 1, 'me': 1, 'now': 1}),
 Counter({'hello': 2, 'you': 1, 'call': 1, 'tomorrow': 1})]


# For now we use the ready implemented Bag of Words-Algo

First deal with test-data

In [40]:
documents = ['Hello, how are you!',
                'Win money, win from home.',
                'Call me now.',
                'Hello, Call hello you tomorrow?']
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()
print(count_vector)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)


In [73]:
count_vector.fit(documents)
count_vector.get_feature_names()

['are',
 'call',
 'from',
 'hello',
 'home',
 'how',
 'me',
 'money',
 'now',
 'tomorrow',
 'win',
 'you']

In [46]:
print("Transform the Vector to a Matrix")
print(count_vector.transform(documents))
print("and put it in an Array")
doc_array = count_vector.transform(documents).toarray()
doc_array

Transform the Vector to a Matrix
  (0, 0)	1
  (0, 3)	1
  (0, 5)	1
  (0, 11)	1
  (1, 2)	1
  (1, 4)	1
  (1, 7)	1
  (1, 10)	2
  (2, 1)	1
  (2, 6)	1
  (2, 8)	1
  (3, 1)	1
  (3, 3)	2
  (3, 9)	1
  (3, 11)	1
and put it in an Array


array([[1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 2, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1]])

Now the Matrix is transformed into an Array. The frequency of every word ist represented by the count of the numbers. To make it easier to read wie transform it into an DataFrame. There we have headers with the single words.

"DataFrame([data, index, columns, dtype, copy])	Two-dimensional size-mutable, potentially heterogeneous tabular data structure with labeled axes (rows and columns)."

In [48]:
freq_matrix = pd.DataFrame(doc_array,
                           columns = count_vector.get_feature_names()
                           )
freq_matrix

,are,call,from,hello,home,how,me,money,now,tomorrow,win,you
0,1,0,0,1,0,1,0,0,0,0,0,1
1,0,0,1,0,1,0,0,1,0,0,2,0
2,0,1,0,0,0,0,1,0,1,0,0,0
3,0,1,0,2,0,0,0,0,0,1,0,1


# Create the Training and Testing Set
Now we use the train_test_split from sklearn. First import the needed function from the framework and than use it to store the four data in separate matrices. (Train and test for message and the label)

In [70]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df['sms_message'],
                                                    df['label'],
                                                    random_state=1,
                                                    test_size = 0.33)
#print("X-Train-Matrix:"); print(X_train); print("X-Test-Matrix:"); print(X_test)
#print("Y-Train-Matrix:"); print(Y_train); print("Y-Test-Matrix:"); print(Y_test)
print('Number of rows in the total set: {}' .format(df.shape[0]))
print('Number of rows in the training set: {}'.format(X_train.shape[0]))
print('Number of rows in the test set: {}'.format(X_test.shape[0]))

Number of rows in the total set: 5572
Number of rows in the training set: 3733
Number of rows in the test set: 1839


# Applying Bag of Words to the data


In [84]:
# Instantiate the CountVectorizer method
count_vector = CountVectorizer()

# Fit the training data and then return the matrix
training_data = count_vector.fit_transform(X_train)

# Transform testing data and return the matrix. Note we are not fitting the testing data into the CountVectorizer()
testing_data = count_vector.transform(X_test)



  (0, 686)	1
  (0, 3035)	1
  (0, 3701)	1
  (0, 4431)	1
  (0, 2734)	1
  (0, 875)	1
  (0, 6710)	1
  (0, 3999)	1
  (0, 6112)	1
  (0, 6171)	1
  (0, 3158)	1
  (0, 1971)	2
  (0, 2921)	2
  (0, 3039)	1
  (0, 956)	1
  (0, 6954)	2
  (0, 3154)	1
  (0, 4203)	1
  (1, 5625)	1
  (1, 5391)	1
  (1, 6242)	1
  (1, 3824)	1
  (1, 5837)	1
  (1, 5663)	1
  (1, 1162)	1
  :	:
  (3731, 2574)	1
  (3731, 237)	1
  (3731, 2397)	1
  (3731, 5442)	1
  (3731, 3995)	1
  (3731, 4638)	1
  (3731, 5162)	1
  (3731, 4242)	1
  (3731, 4181)	1
  (3731, 289)	1
  (3731, 6805)	1
  (3731, 6458)	1
  (3731, 337)	1
  (3731, 4501)	1
  (3731, 3502)	1
  (3731, 6248)	1
  (3731, 1472)	1
  (3731, 3401)	1
  (3731, 6242)	1
  (3731, 5663)	1
  (3731, 6112)	1
  (3732, 1582)	1
  (3732, 5633)	1
  (3732, 6805)	1
  (3732, 3978)	1
